# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
#************************************************************
#**                         Module 6 Challenge             **
#**                                                        **
#** Part II: In this deliverable, I'll use weather data    **
#** skills to plan future vacations. Also, I'll use Jupyter**
#** notebooks, the geoViews Python library, and Geoapify   **
#** API.                                                   **
#**                                                        **
#** The objectives:                                        **
#** 1. Create a map that displays a point for every city   **
#**    in the city_data_df DataFrame (from Part I)         **
#** 2. Narrow down the city data to find my ideal weather  **
#**    condition                                           **
#** 3. For each city in step 2, find the first hotel within**
#**    10,000 meters of the coordinates.                   **
#** 4. Add the hotel name and country in the hover message **
#**     for each city in the map.                           ** 
#************************************************************

In [1]:
#************************************
#** Dependencies and Setup section **
#************************************

import hvplot.pandas
import pandas as pd
import requests
import json
import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [8]:
#*************************************************************************
#** import the city data I created in Part I of this week's challenge.  **
#*************************************************************************

city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.drop(columns=['City_ID'],inplace=True)
city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,constantia,44.1833,28.6500,23.38,58,41,4.54,RO,1711986893
1,udachny,66.4167,112.4000,-7.14,94,99,6.58,RU,1711987038
2,albany,42.6001,-73.9662,8.86,64,100,0.45,US,1711987038
3,bethel,41.3712,-73.4140,10.32,57,100,1.54,US,1711986872
4,blackmans bay,-43.0167,147.3167,15.41,93,100,0.45,AU,1711987039


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
#*************************************************
#** Create map that displays a point for every  **
#** city in the city_data_df DataFrame. The size**
#** of each point should be the humidity in each**
#** city.                                       **
#*************************************************

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoEco",
    frame_width = 1000,
    frame_height = 1000,
    size = "Humidity",
    scale = 1.5,
    color = "City"
)
#display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
#*******************************************************
#** In this step, narrow down the cities to those that**
#** fit my weather conditions criteria, and drop any  **
#** results with null values.                         **
#*******************************************************

criteria_max_temp = 35    #not too hot
criteria_min_temp = 30    #not  cold
criteria_max_humid = 30   #not a lot of humidity. Humdity = mosquitos; hell no.

my_city_data_df= city_data_df[(city_data_df['Max Temp']<=criteria_max_temp) & (city_data_df['Max Temp']>=criteria_min_temp) & (city_data_df['Humidity']<=criteria_max_humid)]
my_city_data_clean_df = my_city_data_df.dropna()
city_data_df = pd.read_csv("output_data/cities.csv")
my_city_data_clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,bouar,5.9340,15.5960,32.75,26,95,3.63,CF,1711987040
52,52,sabha,27.0377,14.4283,33.66,7,0,4.91,LY,1711987047
124,124,murzuq,25.9155,13.9184,34.76,6,0,3.86,LY,1711987058
196,196,ad dabbah,18.0500,30.9500,32.06,10,1,3.20,SD,1711987069
202,202,el hadjira,32.6134,5.5126,31.31,16,0,5.55,DZ,1711987070
265,265,mizdah,31.4451,12.9801,34.02,6,0,3.20,LY,1711987081
274,274,zouerate,22.7187,-12.4521,31.96,16,0,2.32,MR,1711987083
339,339,umm ruwaba,12.9061,31.2158,32.66,9,99,3.47,SD,1711987093
375,375,at taj,24.2049,23.2857,30.18,12,0,3.27,LY,1711987100
376,376,al mijlad,11.0333,27.7333,34.19,7,98,2.50,SD,1711987100


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
#******************************************************
#** Create a DataFrame to hold the hotel information,**
#** if any, pertaining to cities in the my list of   **
#** ideal cities.                                    **
#******************************************************

hotel_df = my_city_data_clean_df.copy()
hotel_df["Hotel Name"]=""
hotel_df.drop(columns=['City_ID'],inplace=True)
hotel_df.reset_index(drop=True, inplace=True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,bouar,5.9340,15.5960,32.75,26,95,3.63,CF,1711987040,
1,sabha,27.0377,14.4283,33.66,7,0,4.91,LY,1711987047,
2,murzuq,25.9155,13.9184,34.76,6,0,3.86,LY,1711987058,
3,ad dabbah,18.0500,30.9500,32.06,10,1,3.20,SD,1711987069,
4,el hadjira,32.6134,5.5126,31.31,16,0,5.55,DZ,1711987070,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
#************************************************************
#** In this step, use the Geoapify to find the first hotel **
#** located within 10,000 meters (6.2 miles) of the        **
#** coordinates of each city (from the hotel dataframe).   **
#************************************************************

radius = 10000
params = {"categories" : 'accommodation.hotel',
          "apiKey" : geoapify_key,
         }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    location_data = name_address.json()
    #print(json.dumps(location_data, indent=4, sort_keys= True))
      
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = location_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
bouar - nearest hotel: Mission catholique
sabha - nearest hotel: فندق الكاظم-سبها
murzuq - nearest hotel: فندق باريس - مرزق
ad dabbah - nearest hotel: No hotel found
el hadjira - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
umm ruwaba - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
al mijlad - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
al wasitah - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
ubari - nearest hotel: فندق اوباري


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,bouar,5.9340,15.5960,32.75,26,95,3.63,CF,1711987040,Mission catholique
1,sabha,27.0377,14.4283,33.66,7,0,4.91,LY,1711987047,فندق الكاظم-سبها
2,murzuq,25.9155,13.9184,34.76,6,0,3.86,LY,1711987058,فندق باريس - مرزق
3,ad dabbah,18.0500,30.9500,32.06,10,1,3.20,SD,1711987069,No hotel found
4,el hadjira,32.6134,5.5126,31.31,16,0,5.55,DZ,1711987070,No hotel found
5,mizdah,31.4451,12.9801,34.02,6,0,3.20,LY,1711987081,No hotel found
6,zouerate,22.7187,-12.4521,31.96,16,0,2.32,MR,1711987083,فندق تيرس زمور
7,umm ruwaba,12.9061,31.2158,32.66,9,99,3.47,SD,1711987093,No hotel found
8,at taj,24.2049,23.2857,30.18,12,0,3.27,LY,1711987100,No hotel found
9,al mijlad,11.0333,27.7333,34.19,7,98,2.50,SD,1711987100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
#***************************************************************
#** Now, create a plot of the results from the previous step. **
#** Add hotel name and country information to the hover       **
#** message on the map.                                       **
#***************************************************************

hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    hover_cols = ['City', 'Humidity', 'Hotel Name', 'Country'],
    c = 'City',
    legend =True
    )

# Display the plot
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)